In [1]:
# import relevant modules
import numpy as np
import gdal
import os
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [2]:
# Assign working directory (from/to where the images will be read/write)
os.chdir('C:/Users/SNMACIAS/Documents/CIMMYT/Drone_missions/eBee/TOL/q190912dur/analysis/')

In [ ]:
# Set figure size (scattered plot)
plt.rcParams['figure.figsize'] = (7, 7)

### A. Import and open image

In [ ]:
# Cut to shapefile
img_in = 'C:/Users/SNMACIAS/Documents/CIMMYT/Drone_missions/eBee/TOL/q190912dur/output/q190912dur_transparent_reflectance_all.tif'      # Input file
img_clp = 'test_clip_.tif'                               # Output clipped file
gdal.Warp(img_clp, img_in, dstNodata = -10000, \
          cutlineDSName = 'C:/Users/SNMACIAS/Documents/CIMMYT/Drone_missions/eBee/TOL/q190912dur/analysis/test_clip.shp', \
          cropToCutline = True)

In [ ]:
img_in = 'test_clip_veg_only_2.tif'      # Input file
img_out = 'test_clip_veg_only_2_.tif'      # Output file
gdal.Warp(img_clp, img_in, dstNodata = -10000)

In [ ]:
# use previously clipped mosaic
img_clp = 'test_clip_.tif'

In [ ]:
# Use entire mosaic
img_clp = 'C:/Users/SNMACIAS/Documents/CIMMYT/Drone_missions/eBee/TOL/q190912dur/output/q190912dur_transparent_reflectance_all.tif'

In [ ]:
# Open the file
fid=gdal.Open(img_clp, gdal.GA_ReadOnly)
rows=fid.RasterYSize
cols=fid.RasterXSize

# Read relevant bands and store them as arrays
G=fid.GetRasterBand(1).ReadAsArray()
R=fid.GetRasterBand(2).ReadAsArray()
R_edge=fid.GetRasterBand(3).ReadAsArray()
NIR=fid.GetRasterBand(4).ReadAsArray()
del fid

In [ ]:
NIR.shape, R.shape

### B. Level 1 image processing: deal with homogenization of bands

In [ ]:
# Assign nan
mask1 = R != -10000                 # Boolean: -10000 false, else true
R_a = R*mask1                       # float and 0's
R_a[R*mask1 == 0] = np.nan          # float and nan

mask2 =  NIR != -10000              # Boolean: -10000 false, else true
NIR_a = NIR*mask2                   # float and 0's
NIR_a[NIR*mask2 == 0] = np.nan      # floar and nan

# Eliminate where either in R or NIR there is nan
mask3 = np.where(np.isnan(NIR_a+R_a), True, False)

# Test whether R and NIR have same shape to be able to plot
NIR_a[mask3].shape, R_a[mask3].shape

In [ ]:
# Define soil line
x1, y1 = np.nanquantile(R_a, 0.9995), np.nanquantile(NIR_a, 0.5)         # Bare bright soil (upper right)
x0, y0 = np.nanquantile(R_a, 0.0005), np.nanquantile(NIR_a, 0.0005)      # Bare dark soil (lower left)

m = (y1 - y0)/(x1 - x0)               # Set slope of soil line
c = (-x0/(x1-x0))*(y1-y0)+y0          # Set intercept of soil line

x1, y1, x0, y0, m, c

In [ ]:
# Scatter plot
plt.scatter([x0, x1], [y0, y1], s=100, c='black')    # Points defining soil line
plt.scatter(R_a, NIR_a, c='blue', alpha=0.01)        # Pairs of Red and NIR pixels

plt.legend(loc=0)
plt.xlabel('Red [reflectance]')
plt.ylabel('NIR [reflectance]')
#plt.xlim([0,0.5])
#plt.ylim([0,0.8])

### C. Level 2 image processing: isolate soil pixels

In [ ]:
# Isolate soil pixels
mask4 = NIR_a < m*R_a + 7*c           # boolean: true: below soil line, false, above soil line
NIR_b = NIR_a*mask4                   # float and 0's
NIR_b[NIR_a*mask4 == 0] = np.nan      # float and nan

R_b = R_a*mask4                       # float and 0's
R_b[R_a*mask4 == 0] = np.nan          # float and nan

In [ ]:
# Scatter plot
plt.scatter([x0, x1], [y0, y1], s=100, c='black')    # Points defining soil line
plt.scatter(R_b, NIR_b, c='blue', alpha=0.01)        # Pairs of Red and NIR pixels

plt.legend(loc=0)
plt.xlabel('Red_soil [reflectance]')
plt.ylabel('NIR_soil [reflectance]')

### D. Level 3 image processing: create mask image with vegetation only

In [ ]:
# Prepare image
indataset = gdal.Open(img_clp)
driver = gdal.GetDriverByName("GTiff")                     # Define driver for image creation
dType = gdal.GDT_Float32

projection = indataset.GetProjection()                     # Read projection
geolocation = indataset.GetGeoTransform()                  # Read georeference

In [ ]:
# Calculate gc and write file
outArray = indataset.GetRasterBand(1).ReadAsArray()
outArray = mask4                                           # Every band

img_out = 'veg_only_maskxx.tif'
outdataset = driver.Create(img_out, cols, rows, 1, dType)  # Image creation
outdataset.SetProjection(projection)                       # Assign projection
outdataset.SetGeoTransform(geolocation)                    # Assign geolocation

outdataset.GetRasterBand(1).WriteArray(outArray)           # Write gc file

### Appendix

In [ ]:
# Assign nan
Rnan =  R != -10000         # Boolean: -10000 false, else true
R[~Rnan] = np.nan           # false to nan, true to float

NIRnan =  NIR != -10000     # Boolean: -10000 false, else true
NIR[~NIRnan] = np.nan       # false to nan, true to float

# Eliminate where either in R or NIR there is nan
mask = np.where(np.isnan(R+NIR), np.where(np.isnan(R), NIR, R), R+NIR)

# Test whether R and NIR have same shape to be able to plot
NIR_a[np.isnan(mask)].shape, R_a[np.isnan(mask)].shape

#NIRnan2 =  NIR < y1*1.05
#NIRsoil = np.copy(NIR)

#Rnan2 =  R < x1*1.05
#Rsoil = np.copy(R)

outArray = np.divide(PVIallpix,PVIfcp)                     # GC calculation (eq. 7 Mass & Rajan 2008)

# Apply final mask reflectance image
G_soil = G*mask4                     # float and 0's
R_soil = R*mask4                     # float and 0's
R_edge_soil = R_edge*mask4           # float and 0's
NIR_soil = NIR*mask4                 # float and 0's

G_soil[G*mask4 == 0] = np.nan              # float and nan
R_soil[R*mask4 == 0] = np.nan              # float and nan
R_edge_soil[R_edge*mask4 == 0] = np.nan    # float and nan
NIR_soil[NIR*mask4 == 0] = np.nan          # float and nan